<a href="https://colab.research.google.com/github/Srdcf/HAR/blob/main/modeloHAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import required libraries
import pandas as pd
import numpy as np
import seaborn as sns

import torch

from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split



In [5]:
#df = pd.read_csv('/content/drive/MyDrive/dataset1.csv')
df = pd.read_csv('/content/drive/MyDrive/dataset2.csv')


In [ ]:
#df.drop('target', axis=1, inplace=True)

In [6]:
df.to_csv('/content/drive/MyDrive/dataset2.csv')

In [7]:
df

,Unnamed: 0,Latitude Change,Longitude Change,Altitude Change,Horizontal Accuracy Change,Speed Change,Gyroscope X,Gyroscope Y,Gyroscope Z,Acceleration X Change,Acceleration Y Change,Acceleration Z Change,Roll (º),Pitch (º),Yaw (º),Magnetometer X Change,Magnetometer Y Change,Magnetometer Z Change,DataLabel
0,0,0.0,0.0,0.0,0.0,0.0,-0.063781,0.059786,0.006924,0.000000,0.000000,0.000000,-19.75767,-11.26569,0.944726,0.000000,0.00000,0.00000,idle
1,1,0.0,0.0,0.0,0.0,0.0,-0.042476,0.115178,0.101730,0.000000,0.000000,0.000000,-19.66945,-11.03860,0.851585,0.000000,0.00000,0.00000,idle
2,2,0.0,0.0,0.0,0.0,0.0,0.053395,0.148067,0.123034,0.000000,0.000000,0.000000,-19.68774,-10.73308,0.631760,0.000000,0.00000,0.00000,idle
3,3,0.0,0.0,0.0,0.0,0.0,-0.029693,0.114113,0.088947,0.000000,0.000000,0.000000,-19.80198,-10.73826,0.417986,0.000000,0.00000,0.00000,idle
4,4,0.0,0.0,0.0,0.0,0.0,-0.072303,0.098135,0.113447,0.000000,0.000000,0.000000,-19.72018,-10.74029,0.188188,0.000000,0.00000,0.00000,idle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119860,119860,0.0,0.0,0.0,0.0,0.0,-0.023435,-0.042742,0.042876,0.172020,0.885110,-0.186050,0.00000,0.00000,0.000000,-1.199999,-0.88125,0.80625,idle
119861,119861,0.0,0.0,0.0,0.0,0.0,-0.056324,-0.040612,-0.006125,0.453840,0.153720,-0.135421,0.00000,0.00000,0.000000,-1.331250,-0.95625,0.91875,idle
119862,119862,0.0,0.0,0.0,0.0,0.0,-0.037150,0.020106,-0.095605,-0.634095,-0.175680,-0.034160,0.00000,0.00000,0.000000,-0.843749,-1.01250,0.50625,idle
119863,119863,0.0,0.0,0.0,0.0,0.0,0.005326,0.062716,-0.018908,-0.293410,-0.297985,-0.127490,0.00000,0.00000,0.000000,-0.450000,-0.76875,0.31875,idle


In [8]:
# Encode categorical labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['DataLabelEncoder'] = label_encoder.fit_transform(df['DataLabel'])

In [9]:
# Get the mapping of original classes to their corresponding numerical values
class_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

print("Class to Label mapping:")
print(class_mapping)


Class to Label mapping:
{'Moving': 0, 'idle': 1, 'picking': 2}


In [ ]:
# Define the lag
lag = 50

# Compute change in GPS features with the specified lag
df['Latitude Change'] = df['Latitude (º)'].diff(periods=lag).fillna(0)
df['Longitude Change'] = df['Longitude(º)'].diff(periods=lag).fillna(0)
df['Altitude Change'] = df['Altitude (m)'].diff(periods=lag).fillna(0)
df['Horizontal Accuracy Change'] = df['Horizontal Accuracy (m)'].diff(periods=lag).fillna(0)
df['Speed Change'] = df['Speed (m/s)'].diff(periods=lag).fillna(0)
df['Acceleration X Change'] = df['Acceleration X (m.s2)'].diff(periods=lag).fillna(0)
df['Acceleration Y Change'] = df['Acceleration Y (m.s2)'].diff(periods=lag).fillna(0)
df['Acceleration Z Change'] = df['Acceleration Z (m.s2)'].diff(periods=lag).fillna(0)
df['Magnetometer X Change'] = df['Magnetometer X (ut)'].diff(periods=lag).fillna(0)
df['Magnetometer Y Change'] = df['Magnetometer Y (ut)'].diff(periods=lag).fillna(0)
df['Magnetometer Z Change'] = df['Magnetometer Z (ut)'].diff(periods=lag).fillna(0)

In [10]:

# Split the data into features and target

#.join(df.loc[:, 'Gyroscope X':'Gyroscope Z'])
#.join(df.loc[:, 'Acceleration X (m.s2)':'Acceleration Z (m.s2)'])
#.join(df.loc[:, 'Magnetometer X (ut)':'Magnetometer Z (ut)'])
X = (df.loc[:, 'Latitude Change':'Speed Change']).join(df.loc[:, 'Gyroscope X':'Gyroscope Z']).join(df.loc[:, 'Acceleration X Change':'Acceleration Z Change']).join(df.loc[:, 'Roll (º)':'Yaw (º)']).join(df.loc[:, 'Magnetometer X Change':'Magnetometer Z Change'])

y = df['DataLabelEncoder']  # Target

In [11]:
X


,Latitude Change,Longitude Change,Altitude Change,Horizontal Accuracy Change,Speed Change,Gyroscope X,Gyroscope Y,Gyroscope Z,Acceleration X Change,Acceleration Y Change,Acceleration Z Change,Roll (º),Pitch (º),Yaw (º),Magnetometer X Change,Magnetometer Y Change,Magnetometer Z Change
0,0.0,0.0,0.0,0.0,0.0,-0.063781,0.059786,0.006924,0.000000,0.000000,0.000000,-19.75767,-11.26569,0.944726,0.000000,0.00000,0.00000
1,0.0,0.0,0.0,0.0,0.0,-0.042476,0.115178,0.101730,0.000000,0.000000,0.000000,-19.66945,-11.03860,0.851585,0.000000,0.00000,0.00000
2,0.0,0.0,0.0,0.0,0.0,0.053395,0.148067,0.123034,0.000000,0.000000,0.000000,-19.68774,-10.73308,0.631760,0.000000,0.00000,0.00000
3,0.0,0.0,0.0,0.0,0.0,-0.029693,0.114113,0.088947,0.000000,0.000000,0.000000,-19.80198,-10.73826,0.417986,0.000000,0.00000,0.00000
4,0.0,0.0,0.0,0.0,0.0,-0.072303,0.098135,0.113447,0.000000,0.000000,0.000000,-19.72018,-10.74029,0.188188,0.000000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119860,0.0,0.0,0.0,0.0,0.0,-0.023435,-0.042742,0.042876,0.172020,0.885110,-0.186050,0.00000,0.00000,0.000000,-1.199999,-0.88125,0.80625
119861,0.0,0.0,0.0,0.0,0.0,-0.056324,-0.040612,-0.006125,0.453840,0.153720,-0.135421,0.00000,0.00000,0.000000,-1.331250,-0.95625,0.91875
119862,0.0,0.0,0.0,0.0,0.0,-0.037150,0.020106,-0.095605,-0.634095,-0.175680,-0.034160,0.00000,0.00000,0.000000,-0.843749,-1.01250,0.50625
119863,0.0,0.0,0.0,0.0,0.0,0.005326,0.062716,-0.018908,-0.293410,-0.297985,-0.127490,0.00000,0.00000,0.000000,-0.450000,-0.76875,0.31875


In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
input_shape = X.shape[1]
model = models.Sequential([
    layers.Dense(512, activation='relu', input_shape=(input_shape,)),
    layers.Dropout(0.4),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(3, activation='softmax')
])

initial_learning_rate = 0.001
opt = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)
def lr_schedule(epoch):
    return initial_learning_rate * 0.95 ** epoch

lr_scheduler = LearningRateScheduler(lr_schedule)

model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X, y, epochs=70, batch_size=128, callbacks=[lr_scheduler])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/70
937/937 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.5017 - loss: 2.0570 - learning_rate: 0.0010
Epoch 2/70
937/937 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.6073 - loss: 0.8043 - learning_rate: 9.5000e-04
Epoch 3/70
937/937 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.6224 - loss: 0.7622 - learning_rate: 9.0250e-04
Epoch 4/70
937/937 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.6281 - loss: 0.7414 - learning_rate: 8.5737e-04
Epoch 5/70
937/937 ━━━━━━━━━━━━━━━━━━━━ 19s 11ms/step - accuracy: 0.6345 - loss: 0.7293 - learning_rate: 8.1451e-04
Epoch 6/70
937/937 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.6422 - loss: 0.7160 - learning_rate: 7.7378e-04
Epoch 7/70
937/937 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.6456 - loss: 0.7095 - learning_rate: 7.3509e-04
Epoch 8/70
937/937 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.6462 - loss: 0.7046 - learning_rate: 6.9834e-04
Epoch 9/70
937/937 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.6503 - loss:

In [13]:
model.evaluate(X_test, y_test, batch_size=128, verbose=1)

NameError: name 'X_test' is not defined

In [ ]:
# Initialize lists to store evaluation results
losses = []
accuracies = []
correct_predictions = 0

for i in range(len(X_test)):
    # Evaluate the current sample
    loss, accuracy = model.evaluate(X_test[i:i+1], y_test.iloc[i:i+1], verbose=0)

    # Append the loss and accuracy
    losses.append(loss)
    accuracies.append(accuracy)

    # Predict the label for the current sample
    prediction = model.predict(X_test[i:i+1], verbose=0)

    # Check if the prediction is correct
    if prediction.argmax() == y_test.iloc[i].argmax():  # Removed extra indent here
        correct_predictions += 1

    # Train the model on the current sample
    model.train_on_batch(X_test[i:i+1], y_test.iloc[i:i+1])

    # Print the results for the current sample
    print(f"Sample {i+1}: Loss = {losses[i]}, Accuracy = {accuracies[i]}")

# Calculate the percentage of correct predictions
percentage_correct = (correct_predictions / len(X_test)) * 100

print(f"Percentage of correct predictions: {percentage_correct:.2f}%")

Sample 1: Loss = 0.7448555827140808, Accuracy = 1.0
Sample 2: Loss = 1.4129648208618164, Accuracy = 0.0
Sample 3: Loss = 0.29115408658981323, Accuracy = 1.0


Sample 4: Loss = 0.6059539914131165, Accuracy = 1.0


Sample 5: Loss = 0.7140113711357117, Accuracy = 1.0
Sample 6: Loss = 0.6458653211593628, Accuracy = 1.0
Sample 7: Loss = 0.12740904092788696, Accuracy = 1.0
Sample 8: Loss = 0.4114813506603241, Accuracy = 1.0
Sample 9: Loss = 0.6383304595947266, Accuracy = 1.0
Sample 10: Loss = 5.686121585313231e-05, Accuracy = 1.0
Sample 11: Loss = 0.6639337539672852, Accuracy = 1.0
Sample 12: Loss = 1.0066252946853638, Accuracy = 0.0
Sample 13: Loss = 0.21373838186264038, Accuracy = 1.0
Sample 14: Loss = 1.0043787956237793, Accuracy = 0.0
Sample 15: Loss = 0.38047996163368225, Accuracy = 1.0
Sample 16: Loss = 0.8480755686759949, Accuracy = 0.0
Sample 17: Loss = 0.6328964829444885, Accuracy = 1.0
Sample 18: Loss = 1.4013720750808716, Accuracy = 0.0
Sample 19: Loss = 0.7383908033370972, Accuracy = 0.0
Sample 20: Loss = 1.2535326480865479, Accuracy = 0.0
Sample 21: Loss = 0.8411253690719604, Accuracy = 0.0
Sample 22: Loss = 2.557854652404785, Accuracy = 0.0
Sample 23: Loss = 0.9980297088623047, Accuracy

KeyboardInterrupt: 

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Get predictions from the model
y_pred = np.argmax(model.predict(X_test), axis=1)

# Create confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
print(conf_matrix)


815/815 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
Confusion Matrix:
[[11443    15  2174]
 [ 2075   364   123]
 [ 6188     2  3686]]


In [4]:
import numpy as np

# Matriz de confusão original
confusion_matrix = np.array([
    [38696, 50, 6614],
    [5588, 2257, 760],
    [18595, 25, 14315]
])

# Calcular a soma total de todos os elementos da matriz
total_sum = np.sum(confusion_matrix)

# Converter cada valor para porcentagem
confusion_matrix_percent = (confusion_matrix / total_sum) * 100

# Arredondar os valores para 2 casas decimais e garantir que não use notação científica
confusion_matrix_percent_rounded = np.round(confusion_matrix_percent, 2)

# Formatar a matriz para exibir os valores corretamente
np.set_printoptions(suppress=True, precision=2)

print("Matriz de Confusão em Porcentagens:")
print(confusion_matrix_percent_rounded)

Matriz de Confusão em Porcentagens:
[[44.53  0.06  7.61]
 [ 6.43  2.6   0.87]
 [21.4   0.03 16.47]]


In [ ]:
model.save('fulldata_porto.h5')

In [ ]:
pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.8 MB/s eta 0:00:00


In [ ]:
pip install tf2onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.


In [ ]:
import tensorflow as tf

# Carregar o modelo Keras
modelo = tf.keras.models.load_model('/content/fulldata_porto.h5')

# Converter o modelo Keras para ONNX
tf.saved_model.save(modelo, "/content/drive/MyDrive/saved_model")
!python -m tf2onnx.convert --saved-model /content/drive/MyDrive/saved_model --output /content/drive/MyDrive/fulldata_porto.onnx


2025-01-13 16:21:31.246157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-13 16:21:31.438456: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-13 16:21:31.462032: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-13 16:21:35.154289: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2